XXX Docker 项目启动会
===

这是 2015-2016 年在 xxx 公司启动 docker 容器平台项目的 kickoff 会。
简单介绍了 docker 的一些技术细节和生态链。

涉及的都是 2016 年当时的技术，这些技术在现在看来有些可能已经过时了。
仅作参考

  - Docker 概念介绍
  - 我们现在的架构
  - 引入 Docker 后的初步设计方案
  - 需要解决的问题

Docker 概念介绍
===

  - Docker 是什么
  - Docker 原理简述
  - Docker 使用概述
  - Docker 能解决什么问题
  - Docker 还有什么问题

## Docker 是什么

> “Docker 是一個開源專案，誕生於 2013 年初，最初是 dotCloud 公司內部的一個業餘專案。它基於 Google 公司推出的 Go 語言實作。 專案後來加入了 Linux 基金會，遵從了 Apache 2.0 協議，原始碼在 GitHub 上進行維護。”

Docker 基于 lxc 容器技术，依赖于 namespace 和 Devicemapper 实现进程应用环境隔离。而不需要传统虚拟机那样的依赖于虚拟系统的环境隔离，从资源上轻了数个数量级，单机部署上千的容器以及灵活的秒即启动都不再是幻想。



![](images/stru.png)

虚拟机是在 Host 之上，再运行了一套完整的虚拟系统。

而 docker 是对 Host 的资源进行更细致的划分，所有的进程其实都直接运行在宿主机上，实现了最大化的资源利用率。

## Docker 原理简述

牛吹完了，下面非常简要的介绍一下 docker 的几项关键技术，这对于增进使用用法的理解非常有用。

  - 隔离
  - 文件系统
  - 网络
  - volume

### namespace & cgroup

容器内的进程需要一套与宿主机隔离的运行环境，Linux 内核中的 namespace 实现了六项隔离参数：

  - UTS 主机名与域名
  - IPC 信号量
  - PID 进程编号
  - Network 网络栈
  - Mount 挂载点
  - User 用户
  

除了运行环境隔离外，容器还需要物力资源隔离，这一特性是依赖于 cgroups 来实现的。

cgroups 最初名为 process container，是由 Google 于 2006 年 提出，于 2007 年重命名为 contril groups 并整合进 Linux 内核之中。

顾名思义，cgroups 是 Linux 内核的一种机制，这种机制可以根据需求将一系列系统任务（进程或线程）划分到不同的组中，并对各组进行资源管理。

cgroups 提供了四种维度的资源管理：

  - 资源限制：可以限制内存使用量等；
  - 优先级管理：可以控制 CPU 时间片和磁盘 IO 的分配，来调控优先级；
  - 资源统计：可以统计任务组的资源使用量；
  - 任务控制：可以对任务组内的任务进行挂起、恢复等操作。

上面介绍了这么多，只是为了让我们能够对 docker 的原理有一个大概的认识，其实在使用中我们并不需要去直接的操作 namespace 或 cgroup，我们只需要理解

**docker 就是利用 namespace & cgroup 实现了资源的隔离，其实进程依然运行在宿主机上，docker 与宿主机使用同一个进程，但是却拥有完全独立的运行环境，正是这一特性造就了 docker 超强的效率**

### 文件系统

docker 容器使用了联合挂载（aufs）和设备映射（DeviceMapper）技术来实现文件系统。

具体的实现细节就不介绍了，只介绍一下最后的表现形式。



![](images/filesystem.jpg)

  - 文件系统由 bootfs 逐层向上创建，每一层都包含各自的文件，且都是只读的；
  - 容器构建完毕后，最后在最上层加上一层可读写层；
  - 用户读取文件时，从上向下检索；
  - 用户写文件时，写在最上方的客读写层上。

这些知识点在你层层构建镜像或 link volume 的时候可能会考虑到，这里就先介绍到这里了

### 网络

Docker 默认使用桥接模式：

  - 启动 docker daemon 后，会在宿主机中创建一个 docker0 网卡，地址为 172.17.0.0/16；
  - 启动容器后，会在容器内创建 eth0 网卡，通过 veth-pair 连接到 docker0，地址为 172.17.0.1/16;
  
容器可以与宿主机外连接，容器与容器可以通过 bridge0 连接。

![](images/bridge.jpg)

#### 网络配置

docker 目前原生支持四种网络模式，可以通过 `--net` 设置：

  - bridge 默认的桥接模式；
  - host 不使用 network namespace，直接处于宿主机的网络环境之中；
  - container 指定一个容器，与之共享同一个 network space；
  - none 不配置容器的网络配置。

#### 常用的网络参数

一般在实践中常会用到两个参数：

  - `-p, --publish` 端口绑定，<host_port>:<container_port> 将容器端口与宿主机端口绑定；
  - `--link`：通过修改 iptables 来实现连接到指定容器
  
  
顺带提一下，docker 默认是允许容器互联的，而且容器也是默认对所有 ip 开放的，网络安全方面需要额外的措施。

#### 跨主机通讯

目前跨主机通讯有两大主流流派，macvlan 和 overlay

  - macvlan 就是单网卡多 IP，将容器以独立 IP 接入到宿主机所在的网段之中；
  - overlay 就是隧道技术，如 vxlan 等。
  
考虑到我们的需求，我觉得 macvlan 比较简单，而且比较符合我们的需求。

### Volume

docker 中还引入了存储卷（volume）的概念，当你创建容器的时候指定了 `--volume` 命令，就会在创建容器的同时创建一个 volume。

volume 都存放在 `/var/lib/docker/volumes`，volume 的生命周期独立于容器，除非你显式的删除，否则 volume 会一直放在这，即使容器被删除也不会影响到 volume。

有几种方法可以创建 volume：

  - `-v <container_path>` 指定容器内某路径创建 volume
  - `-v <host_path>:<container_path>` 将宿主机某路径挂载到容器内，并创建 volume
  - `--volumes-from <container>` 与指定容器共享 volume

## Docker 使用概述

  - Dockerfile
  - images & registry & dockerhub
  - container

### Dockerfile

Dockerfile 是 docker 的核心，可以从零到有生成一个镜像。

Dockerfile 里的每一个命令行都会生成一层 layer，一层层的生成最终的镜像。

在反复的构建中，可以尽可能的保存和重用 layer，加快构建速度。

### images & registry & dockerhub

镜像间可以互相依赖，增加重用率。

官方提供 dockerhub 用于保存 Dockerfile 和镜像，并且支持自动构建，速度也还凑合。

也可以在本地搭建 docker registry，构建完的镜像可以 push 到本地的 registry 内，方便取用。

### container

镜像就像是虚机的 snapshot，可以根据镜像生成任意多的容器。

而且镜像在生成容器的时候，还可以指定运行参数和运行命名。

容器可以启动、停止和销毁。

容器在改动后可以 commit 为新的镜像。

## Docker 能解决的问题

 - 环境的一致性（从开发到测试到运维部署，环境绝对一致）
 - 多样性的环境需求
 - 多样性的语言需求
 - 更低的额外开销（虚机系统开销）
 - 快速部署

## Docker 还存在的问题

  - 完全将过去的模式推倒重来，生态圈还在健全中；
  - 安全性（时不时会暴露出一些安全性 bug，所以最好保持使用最新版本）
  - 资源限制，docker 并未实现全部的 namespace，所以部分资源限制功能并不完善

Docker 当前的生态链
===

* 镜像管理
* 监控
* 网络
* 日志
* 资源管理
* 编排系统

![](./images/docker-environment.jpg)

Kubernetes
===

在我们后期的 v2.0 项目中，底层全部采用 kubernetes v1.2，所以稍微介绍一下 k8s。


![](./images/k8s.jpg)

k8s 脱胎于 Google 内部的 borg 项目。我们之前提到过，docker 的核心技术其实在很多年前就已经出现了，而作为科技龙头的谷歌当然也很早就注意到了该技术，并且在内部的生产服务器上大规模的应用。

近年来随着 docker 的火爆，google 也成立了团队，结合 borg 的使用经验，开发并维护了一款以 docker 为基础的编排系统：kubernetes。


## Kubernetes Roles


要了解 kubernetes，需要先了解 k8s 体系中的角色：

* Pod
* Replication Controller (RC)
* Service (SVC)
* Deployment

Pod 是 k8s 对容器的抽象，最小的执行单元，负责定义每一个容器的具体属性。

RC 是 k8s 对与 pod 的更高层封装，可以对 pod 进行编排（比如一个 rc 启动多个 pod 来实现水平扩容）。

SVC 是服务的最小单元，每一个 SVC 都会在 k8s 的内部网络中暴露一个 cluster ip，这个 ip 是随着 SVC 的生命周期而持续的。

SkyDNS 是 k8s 内部的解析服务，可以将 service-name 解析为 cluster ip

Deployment 是新版 k8s 对 Pod 和 RC 的封装，可以直接在 deployment 内定义和 pod、svc 相关的配置，并且自动实现滚动升级等编排特性。

![](./images/k8s_framework.jpg)

我们现在的架构
===



![](images/current_stru.png)

目前：

  - 虚机运行在宿主机的网段中
  - 手动发包部署
  - 发布后，手动修改 UCM 配置

所以，其实我第一期的目标，就是将现有架构中的虚拟机部分，换成 docker

听上去很简单，但是万事开头难，要完成这一目标就意味着我们要搭建起一整套的适用于 docker 的架构。为后期的扩展做好准备。

引入 Docker 后的架构（初步设计）
===

![](images/future_stru.png)

可以看到前半程的架构几乎不变，主要的变动在 **FTP —— App** 这一段。

当用户在 FTP 上打好包后，自动触发 docker builder 进行镜像打包，然后运维人员在 Messer 上控制部署。

我们的解决方案
===

  - 自动构建的 builder
  - 镜像管理
  - 容器部署
  - 网络
  - 容器监控
  - 日志
  - 容器排错

v1.0 版的解决方案
===

用户提交发包申请后，自动开始拉取 FTP 并打包成镜像，等待运维人员部署。

各机器上的镜像在每次部署后都无差别的清理一次。

运维人员在 Messer 上点击界面，选择指定镜像和目标 IP，然后一键部署。

网络采用 macvlan，直接接入所在网段

可以继续使用现有的 zabbix 对 APP 和宿主机进行监控

日志使用 log-driver=syslog 写到 `/var/log/message` 内（`docker/<tag>`），然后可以用 rsyslog 做策略

使用 `docker exec` 可进入到容器进行排错

v2.0 版的解决方案
===

![](./images/docker-pj-framework.jpg)

In [ ]:
在保持原有发布系统不变的基础上，增加了一套完全基于容器的构建系统和发布系统。

## DaoCloud

了解了我们的需求和问题，再来看看 daocloud。

daocloud 主要的解决方案集中在 PaaS 上，而不是私有云的部署方案，整体解决办法支持不足。

合理的利用 daocloud webhooks 和 docker-compose 也可以实现我们的需求，不过每次构建都需要从外网（daocloud 服务器）拉取镜像，对带宽是一个不小的压力。

要解决以上两个问题，都需要我们投入开发力量，而且还需要 daocloud 的人员进行一些定制化的修改，由于我方基本确定了以私有云为主的发展方向，是否依赖于 daocloud 需要仔细考量。


## 工作安排

  0. 首先熟悉 docker 的基本操作，我稍后会整理一份常用操作文档
  0. 有兴趣有精力的人可以报名一下，领一个感兴趣的部分去研究、尝试一下
  0. 最好是有几个人一人攻克一项，不然就一起一项一项的搞定
  0. 如果手头有其他活要忙但是也感兴趣的话，建议也可以参加，帮忙提供某方面的资料，或提出建议都可以